In [1]:
%load_ext dotenv
%dotenv

In [2]:
from langchain_openai.chat_models import ChatOpenAI
chat = ChatOpenAI(model_name = 'gpt-4.1-mini', seed = 365, temperature=0, max_tokens = 100)

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import CommaSeparatedListOutputParser

In [4]:
list_instructions = CommaSeparatedListOutputParser().get_format_instructions()

In [5]:
print(list_instructions)

Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [6]:
chat_template = ChatPromptTemplate.from_messages([
    ('human', "I've recently adopted a {pet}. Could you suggest three {pet} names? \n" + list_instructions)
])

In [7]:
print(chat_template.messages[0].prompt.template)

I've recently adopted a {pet}. Could you suggest three {pet} names? 
Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`


In [8]:
chain = chat_template | chat | CommaSeparatedListOutputParser()

In [9]:
result = chain.invoke({'pet': 'dog'})

In [10]:
print(result)

['Buddy', 'Luna', 'Max']


## Batching

Faster, run in parallel.

In [11]:
chain.batch([
    {'pet': 'fish'},
    {'pet': 'cat'},
    {'pet': 'tiger'}
])

[['Bubbles', 'Nemo', 'Finley'],
 ['Whiskers', 'Luna', 'Milo'],
 ['Shere Khan', 'Rajah', 'Tigris']]

## Streaming

This is more memory efficient. They are single use. Once you read them to the end, they cannot be reset to start reading again from the beginning.

In [12]:
response = chain.stream({'pet': 'leopard'})

In [14]:
for token in response:
    print(token, end = '')

['Luna']['Blaze']

In [21]:
response = chat.stream('Teach me a bit about the country Kazakhstan')

In [22]:
for token in response:
    print(token.content, end = '')

Certainly! Here’s an overview of Kazakhstan:

**Location and Geography:**  
Kazakhstan is a large country in Central Asia and is the world's ninth-largest country by land area. It is landlocked, bordered by Russia to the north, China to the east, Kyrgyzstan, Uzbekistan, and Turkmenistan to the south, and the Caspian Sea to the west. The terrain includes vast steppes, deserts, mountains, and lakes.

**Capital and Major Cities:**  
The capital city